In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 

In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table', {'class':'wikitable sortable'}).tbody  # Obtained from inspect (wiki - right click)

In [3]:
rows = table.find_all('tr')
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]

In [4]:
df = pd.DataFrame(columns = columns)

In [15]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')

    if len(tds) == 3:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', '')]
    else:
        values = [td.text for td in tds]
    df = df.append(pd.Series(values, index=columns), ignore_index = True)

In [17]:
# Getting the names of the indexes for which the Borough has a value - Not assigned
indexNames = df[df['Borough'] == 'Not assigned'].index
# Delete these rows index from the Data Frame
df.drop(indexNames, inplace = True)

In [18]:
# Replacing the Neighbourhood = "Not Assigned" to Borough values
df.loc[(df.Neighbourhood == 'Not assigned'), 'Neighbourhood'] = df['Borough']

In [19]:
# Grouping different neighbourhood based on the postcode.
df = df.groupby('Postcode').agg({'Borough':'first', 
                                    'Neighbourhood': ','.join}).reset_index()

In [20]:
df.to_csv('C:/Users/ragha/Desktop/Projects/PostalCode_Canada.csv', index = False)



In [21]:
print(df.shape)

(103, 3)
